# Data Programming in Python  - Group Project Report - Group 7

# Business Analytics Graduate Program: MSCI: 6040:0EXA\0EXF Smr20

***

# Project Contents:

## Folder Structure and contents:

| Folder Name | File Name | Description |
| :- | :- | :- |
| Group7Data | Group7_File1_DataValidation_and_Preprocessing.ipyb | File1 containing Validation and preprocessing steps
| Group7Data | Group7_File2_Machine Learning and Prediction.ipynb.ipyb | File2 containing model building steps
| Group7Data | Group7_File2_Machine Learning and Prediction.html | HTML File for File1
| Group7Data | Group7_File2_Machine Learning and Prediction.html | HTML File for File2
| Group7Data | Group7app.py | Flask application
| Group7Data > data | Product_data.csv | Product Master Data file 
| Group7Data > data | Stores_data.csv | Dealer/Customer Master Data file
| Group7Data > data | Sales_data.csv | Sales data showing sales by customer by product
| Group7Data > models | group7regressionmodel.pkl | Pickle File

***

#### File:1 - Data Validation and Preprocessing
#### File:2 - Machine Learning and Prediction

***

## File 1: Data Validation and Preprocessing

# Notebook Structure

1. Problem Statement and Objective
2. Hypothesis Generation and Data Collection
3. Loading Datasets and Libraries
4. Understanding and Validating the Data
5. Data Exploration - Train, Product, Store 
6. Data Preprocessing 

# 1. Problem Statement and Objective

**Problem Statement:** 
One of the largest office furniture manufacturers in USA produces office furnitures such as desks, seating, storage, workspace products and furniture accessories. These products are sent to a dealer network and then the goods are sold to end customers through stores. Not maintaining the right inventory levels by dealers will result in a sudden influx of orders, where the manufacturer may not able to meet the demand in short lead times. The demand volatility incurred by the dealers results in high business revenue losses to the furniture manufacturer.

**Problem Objective (Goal):**
The main objective of the project is to prevent overstocking and understocking of items by forecasting the demand of furniture items for next week using predictive analytics. Overstocking of furniture goods affects the cash flow of organization and understocking will lead a poor customer experience and led to poor customer loyalty results in missed sales. 


---

# 2. Hypothesis Generation and Data Collection

Hypothesis generation helps to find the possible view or assertion of an analyst about the problem. To help analyse the problem, the data has been collected to predict the target variable. The input data in multiple CSV files is gathered from the Manufacturer. The column and definitions are in the below order:

**Data Description:**

Sales Data - Sales_data.csv 
- **WEEK_END_DATE** - Week ending date
- **STORE_ID** - Store identifier
- **PRODUCT_ID** - Product specific identifier
- **PRICE** - Selling price of item after discount
- **BASE_PRICE** - Base price of item
- **DISPLAY** - Product was a part of in-store promotional display
- **FEATURE** - Product was in in-store circular/flyer
- **UNITS** - Units sold (Target variable in our analysis)

Product Data - Product_data.csv
- **PRODUCT_ID** - Product specific identifier
- **DESCRIPTION**	- Product description
- **MANUFACTURER** - Name of product manufacturer or brand name
- **CATEGORY** - Category of product
- **SUB_CATEGORY** - sub-category of product
- **PRODUCT_WEIGHT_LB** - Product weight measured in LBs

Store Data - Store_data.csv
- **STORE_ID** - Store number
- **STORE_NAME** - Name of store
- **ADDRESS_CITY_NAME** - Name of the City
- **ADDRESS_STATE_PROV_CODE** - Name of the State
- **MSA_CODE** - (Metropolitan Statistical Area) Unique code based on geographic region and population density
- **SEG_VALUE_NAME** - Store segment name
- **PARKING_SPACE_QTY** - Number of parking spaces in the store parking lot
- **SALES_AREA_SIZE_NUM** - Area of store in sqft
- **AVG_WEEKLY_ORDERS** - Average weekly orders at a dealer/store

# 3. Loading Datasets and Libraries

#### Pre-requisite Libraries to be installed to setup environment:
`pip install seaborn`<br>
`pip install pandas`<br>
`pip install numpy`<br>
`pip install category_encoders`<br>
`pip install matplotlib`<br>
`pip install DateTime`<br>
`pip install seaborn`<br>
`pip install sklearn`<br>
`pip install statsmodels`<br>
`pip install scipy`<br>
`pip install flask`<br>
`pip install flask_restful`

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import random
import category_encoders as ce
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

We are provided with three tables containing the required information:

- **Product_data**: Consists of details about the product
- **Store_data**: Consists of details of various dealers/customers associated with the manufacturer  
- **Sales_data**: Contains transaction data of products

In [ ]:
# reading the data files
train = pd.read_csv('data/Sales_data.csv')
store_data = pd.read_csv('data/Store_data.csv')
product_data = pd.read_csv('data/Product_data.csv')

In [ ]:
# checking the size (rowsxcols) of the dataframes to ensure Size of the csv and the df,
train.shape, product_data.shape, store_data.shape

# 4. Understanding and Validating Data

As best coding practice, data validation for each column has to be done . Going through variables one by one to understand what features are available to use in the data sets. The features will be later used to make insights about the data to be able to use it for ML.

### Train Data - Store Data that holds Weekly sales by Dealer

In [ ]:
# Verify to make sure data is loaded properly.
store_data.head()

In [ ]:
# Verify to make sure data is loaded properly.
train.head()

In [ ]:
#Verify to make the loaded properly
product_data.head()

In [ ]:
# checking datatypes of columns in train file
train.dtypes 

#### Inferences:
WEEK_END_DATE has the data type object, but its a datetime variable <br>
The store number and product codes are read as int, represents categorical variables.


##### Validate Week_end_date

In [ ]:
#convert into the date time format
train['WEEK_END_DATE'] = pd.to_datetime(train['WEEK_END_DATE'])

In [ ]:
#check for null values
train['WEEK_END_DATE'].isnull().sum()

In [ ]:
#Verify data to find any extreme values. The data collected is from July 2017 to April 2020
train['WEEK_END_DATE'].min(), train['WEEK_END_DATE'].max()

#### Infereneces:
- The data collected is from July 2017 to April 2020

#### Are any dates missing from this period?

In [ ]:
#Time span for which data is available in train df. The last day is not included so the unique weeks will be 141+1
(train['WEEK_END_DATE'].max() - train['WEEK_END_DATE'].min())/7

In [ ]:
#The training data is for 142 weeks, based on the number of unique *weekend dates* in the train file. Verify using nnunique function.
train['WEEK_END_DATE'].nunique()

#### Inferences:
Every date entry in WEEK_END_DATE represents the last day of the week. Therefore we have 142 weeks.

##### Validate STORE_ID and PRODUCT_ID

In [ ]:
#Is there any store and prodcut_id null. Are there any missing values in the variables?
train[['STORE_ID', 'PRODUCT_ID']].isnull().sum()

In [ ]:
#unique values of store_num
train['STORE_ID'].nunique()

We have 76 unique stores.

In [ ]:
(train['STORE_ID'].value_counts()).sort_values()

Every store has minimum of 1676 transactions.

#### Does each store hold atleast one entry per week?

We have 76 unique stores and 142 weeks of data for the sales. 
If each store is selling occupies atleast one row in the data, the minimum number of unique rows should be 142*76

In [ ]:

142*76

In [ ]:
train[['WEEK_END_DATE','STORE_ID']].drop_duplicates().shape
  

#### Inferences:
Implies that  every store is selling atleast 1 product each week. 

In [ ]:
train['PRODUCT_ID'].nunique()

In [ ]:
(train['PRODUCT_ID'].value_counts()).sort_values()

#### Is every product sold in every week. Check using unique values of week_end_date and prodcut_id columns. 

In [ ]:
train[['WEEK_END_DATE','PRODUCT_ID']].drop_duplicates().shape

In [ ]:
142*30

#### Inferences:
This implies every product is being sold atleast 1 time every week as per the data set.

Assuming we have information for the sale of every product that is present in the product table (30), against each store associated (76), and for every week (142); we should have `142*76*30` data rows.

In [ ]:
#Find the cross product of Weeks*product*store
142*76*30

Is each store selling each product throughout the given period?

In [ ]:
train.shape

In [ ]:
232286/323760

#### Inferences:
We can conclude that all stores are not selling all products each week
of all the possible combinations, about 72% of the data is present

#### For a store selling a particular product, do we have more than one entry?

Each product sold by any store should hold only one row, i.e. a particular store,say 'store A' selling a product 'prod P' should contribute a single row for every week. Validate the theory:

In [ ]:
train.shape

In [ ]:
train[['WEEK_END_DATE','STORE_ID','PRODUCT_ID']].drop_duplicates().shape

In [ ]:
train.groupby(['WEEK_END_DATE','STORE_ID'])['PRODUCT_ID'].count().mean()

**Inferences:**
- The shape does not change after using drop duplicates,
- Implies that there are unique combinations for week, store and Product_id
- On an average, each week we are selling 22 products


#### Is a store selling a product throughout the period or is there a break?

In [ ]:
(train.groupby(['STORE_ID', 'PRODUCT_ID'])['UNITS'].count()).sort_values()

From above result, it is observed that not all stores sell a product throughout the week.
Our total week count is 142, here the min count shows 137


##### Validate BASE_PRICE

In [ ]:
train['BASE_PRICE'].isnull().sum()

In [ ]:
train['BASE_PRICE'].describe()

**Distribution of Base Price variable**

In [ ]:

plt.figure(figsize=(8,6))
sns.distplot((train['BASE_PRICE'].values), bins=50, color = "teal" , kde=True)
plt.xlabel('Price Distribution', fontsize=12)
plt.show()

##### Validate FEATURE and DISPLAY

In [ ]:
train[['FEATURE','DISPLAY']].isnull().sum()

In [ ]:
train[['FEATURE','DISPLAY']].dtypes

In [ ]:
train[['FEATURE','DISPLAY']].nunique()

In [ ]:
train['FEATURE'].value_counts(normalize=True)

about 9-10% product was in display in Stores.

In [ ]:
train['FEATURE'].value_counts(normalize=True).plot(kind='bar',color='mediumturquoise')

Approximately 10 percent of product are featured

In [ ]:
train['DISPLAY'].value_counts(normalize=True)

About 13% product was in display in Stores.

In [ ]:
train['DISPLAY'].value_counts(normalize=True).plot(kind='bar',color='blueviolet')

In [ ]:
pd.crosstab(train['FEATURE'], train['DISPLAY']).apply(lambda r: r/len(train), axis=1)

##### Validate UNITS

In [ ]:
train['UNITS'].isnull().sum()

Basic statistical details of UNITS variable

In [ ]:
train['UNITS'].describe()

#### Inferences:
- The Range of values is very high
- Minimum number of units sold is 0 and maximum is 1800 
- A huge difference between the 75th percentile and the max value indicates presence of outliers


#### Validate maximum units rows from df. From the describe, max units is 1800

In [ ]:
train[train['UNITS'] == 1800]

#### Validate minimum units rows from df. From the describe, min units is 0

In [ ]:
train[train['UNITS'] == 0]

In [ ]:
# scatter plot for UNITS variable
plt.figure(figsize=(8,6))
plt.scatter(x = range(train.shape[0]), y = np.sort(train['UNITS'].values),color='chocolate')
plt.xlabel('Index', fontsize=12)
plt.ylabel('Units Sold', fontsize=12)
plt.show()

**Inferences:**
1. Most of the values are less than 250
2. There are a few entries that are outliers (with 1 outlier way outside the range)

## Product data - Item master showing features of the item and categorization

In [ ]:
# first five rows of product data
product_data.head()

In [ ]:
product_data.dtypes

**Validating PRODUCT_ID**

In [ ]:
product_data['PRODUCT_ID'].nunique()

**Validate if all the product in train data set are in product data set and viceversa**  

In [ ]:
len(set(product_data.PRODUCT_ID).intersection(set(train.PRODUCT_ID)))

#### Inferences:
All products in train data set and product data set matches

**Validating CATEGORY**

In [ ]:
# number and list of unique categories in the product data
product_data['CATEGORY'].nunique(), product_data['CATEGORY'].unique()

In [ ]:
product_data['CATEGORY'].isnull().sum()

In [ ]:
product_data['CATEGORY'].value_counts()

**Validate SUB_CATEGORY**

In [ ]:
product_data['SUB_CATEGORY'].isnull().sum()

In [ ]:
product_data['SUB_CATEGORY'].nunique()

In [ ]:
# displaying subcategories against each category
product_data[['CATEGORY','SUB_CATEGORY']].drop_duplicates().sort_values(by = 'CATEGORY')

 #### Inferences:
 - The sub-categories give additional detail about the product.
    - Accessories has 2 sub-categories (Lighting and Power access)
    - Tables has Cafe tables and Height-adjustable table as subcategories
    - Seating and Storage have just 1 sub category, no further division

##### Validate Brand / MANUFACTURER

In [ ]:
product_data['MANUFACTURER'].isnull().sum()

In [ ]:
product_data['MANUFACTURER'].nunique()

In [ ]:
# displaying the list of manufacturers against the 4 categories
temp = product_data[['CATEGORY','MANUFACTURER']].drop_duplicates()
pd.crosstab([temp['CATEGORY']], temp['MANUFACTURER'])

#### Inferences:
We have 4 unique categories of Products (Accessories, Seating, Storage, Tables)<br>
Each category is associated with more than 1 brand

### Store Data - Customer master that holds all the dealer/customer details

In [ ]:
store_data.head()

In [ ]:
store_data.dtypes

##### Validate STORE_ID and STORE_NAME

In [ ]:
store_data['STORE_ID'].nunique()

#### To Validate if all stores in the train_data is present in store_data

In [ ]:
len(set(store_data.STORE_ID).intersection(set(train.STORE_ID)))

In [ ]:
store_data['STORE_NAME'].isnull().sum()

In [ ]:
#unique store name:
store_data['STORE_NAME'].nunique()

In [ ]:
# number of store names repeating
store_data['STORE_NAME'].value_counts()

#### Inferences:
There are no stores without name in the data set<br>
The number of unique store IDs is more than number of unique store names which implies that there might be stores with same name, located in different city

##### Validate ADDRESS_CITY_NAME  and ADDRESS_STATE_PROV_CODE

In [ ]:
store_data[['ADDRESS_STATE_PROV_CODE', 'ADDRESS_CITY_NAME']].isnull().sum()

#### How many cities and states are the stores located in?

In [ ]:
store_data[['ADDRESS_STATE_PROV_CODE', 'ADDRESS_CITY_NAME']].nunique()

Find out the number of stores in each of the state 

In [ ]:
store_data.groupby(['ADDRESS_STATE_PROV_CODE'])['STORE_ID'].count()

#### Inferences:
Each store has a unique store ID <br>
Most stores are from Ohio and Texas ~93%<br>
Few from Kentucky and Indiana ~7%<br>

In [ ]:
store_data.groupby(['ADDRESS_STATE_PROV_CODE'])['ADDRESS_CITY_NAME'].nunique()

In [ ]:
store_data['ADDRESS_CITY_NAME'].value_counts()

##### Validate MSA_CODE

In [ ]:
store_data['MSA_CODE'].nunique(), store_data['MSA_CODE'].unique()

In [ ]:
store_data['MSA_CODE'].value_counts()

In [ ]:
(store_data.groupby(['MSA_CODE', 'ADDRESS_STATE_PROV_CODE'])['STORE_ID'].count())

#### Inferences:
These codes are assigned based on the geographical location and population density.

##### Validate PARKING_SPACE_QTY  and SALES_AREA_SIZE_NUM 

In [ ]:
store_data[['PARKING_SPACE_QTY', 'SALES_AREA_SIZE_NUM']].isnull().sum()

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(store_data['PARKING_SPACE_QTY'], bins=25, kde=False, color='magenta')
plt.xlabel('Parking Area Size', fontsize=12)
plt.show()

#### Inference:
Out of the 76 stores, parking area of 51 is missing as per our data set
<br>About 20 stores have parking area between 250 - 500 units

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(store_data['SALES_AREA_SIZE_NUM'], bins=30, kde=True,color='darkmagenta')
plt.xlabel('Sales Area Size (Sq Feet)', fontsize=12)
plt.show()

#### Inferences:
- Most stores have the area between 30-70 K
- Only a small number of stores have area less than 30k or greater than 90k

#### How is Average store size varying for different states?

In [ ]:
(store_data.groupby(['ADDRESS_STATE_PROV_CODE'])['SALES_AREA_SIZE_NUM'].mean()).sort_values(ascending=False)

In [ ]:
state_oh = store_data.loc[store_data['ADDRESS_STATE_PROV_CODE'] == 'OH']
state_tx = store_data.loc[store_data['ADDRESS_STATE_PROV_CODE'] == 'TX']

sns.distplot(state_oh['SALES_AREA_SIZE_NUM'], hist=False,color= 'dodgerblue', label= 'OHIO')
sns.distplot(state_tx['SALES_AREA_SIZE_NUM'], hist=False,  color= 'orange', label= 'TEXAS')

- Indiana has only one store and the area size is 58,563 sq feet. 
- Ohio and Texas have average around 52k and 50k. 
- Ohio has stores distributed at all sizes.
- Texas mainly has stores between sales area 30k to 60k 

##### Validate AVG_WEEKLY_ORDERS

In [ ]:
store_data['AVG_WEEKLY_ORDERS'].isnull().sum()

In [ ]:
store_data['AVG_WEEKLY_ORDERS'].describe()

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(store_data['AVG_WEEKLY_ORDERS'], bins=30, kde=True,color='gold')
plt.xlabel('Average Baskets sold per week', fontsize=12)
plt.show()

#### Find the average weekly orders sold for the states? This will show if the sales is concentrated to one area.

In [ ]:
(store_data.groupby(['ADDRESS_STATE_PROV_CODE'])['AVG_WEEKLY_ORDERS'].mean()).sort_values(ascending=False)

##### Validate SEG_VALUE_NAME

In [ ]:
store_data['SEG_VALUE_NAME'].isnull().sum()

There are certain segments assigned to store, based on the brand and quality of products sold at the store.

- **Upscale stores** : Located in high income neighborhoods and offer more high-end product
- **Mainstream stores** : Located in middle class areas, offering a mix of upscale and value product
- **Value stores** : Focus on low prices products targeting low income customers

find out the distribution of stores in each of these segments

In [ ]:
store_data['SEG_VALUE_NAME'].value_counts()

#### Does the segment has any relation with the store area? Is there a difference in the average sales for each segment?

In [ ]:
(store_data.groupby(['SEG_VALUE_NAME'])['SALES_AREA_SIZE_NUM'].mean()).sort_values(ascending=False)

In [ ]:
(store_data.groupby(['SEG_VALUE_NAME'])['AVG_WEEKLY_ORDERS'].mean()).sort_values(ascending=False)

#### Inferences:
Higher segment value has higher sales

# 5. Data Exploration - Sales, Product, Store

## Validating the Hypothesis

As a part of hypothesis generation, we validated the below variables against the data to understand any trend or pattern on the product sales, if any.  

**Analyze and identify if there is a trend of a pattern between these variables:**
- Merging the Store and Product Datasets
- Product sales and Weekend Date (Sales by week)
- Category wise Product sales and Week end date - Weekly product sales
- Dealer, Season and Product Sales
- Featured or Displayed Product and Product sale
- Product price and Product Sales 
- Brand and Product sales
- Store location (State) and product sales
- Dealer Store Size and Product sales

### Data wrangling

#### Merging the Store and Product Datasets

In [ ]:
store_product_data = train.merge(product_data, how = 'left', on='PRODUCT_ID')

store_product_data = store_product_data.merge(store_data, how = 'left', on = 'STORE_ID')

In [ ]:
#verify
store_product_data.shape

In [ ]:
store_product_data.columns

### Product sales and Weekend Date (Sales by week)
Is there a trend on  sales or demand patterns over the period of time given in data set?

In [ ]:
#sum of units sold per week
weekly_demand = store_product_data.groupby(['WEEK_END_DATE'])['UNITS'].sum()

plt.figure(figsize=(30,10))
sns.lineplot(x = weekly_demand.index, y = weekly_demand)

#### Inferences:
- Displays the total number of units sold by the dealer (including all products and from all stores)
- The highest number is close to 80,000 and lowest is close to 20,000 units
- There is no evident pattern or trend in the plot
- The spikes can be seen in either direction and at no constant interval

### Category wise Product sales and Week end date - Weekly product sales

Is there a trend on category wise sales or demand patterns or any similarity within each category

In [ ]:
# function to plot weekly sales of products
def product_plots(product_list):
    
    # dictionary storing UPC and weekly sales
    d = {product: store_product_data[store_product_data['PRODUCT_ID'] == product].groupby(['WEEK_END_DATE'])['UNITS'].sum() for product in product_list}
    fig, axs = plt.subplots(len(product_list), 1, figsize = (20, 20), dpi=300)
    j = 0
    
    for product in d.keys():
        # adding manufacturer and descritption in title
        manu = product_data[product_data['PRODUCT_ID'] == product]['MANUFACTURER'].values[0]
        desc = product_data[product_data['PRODUCT_ID'] == product]['DESCRIPTION'].values[0]            
        # creating the plot
        sns.lineplot(x = d[product].index, y = d[product],ax = axs[j]).set_title(str(manu)+str(" ")+str(desc), y=0.75, fontsize = 16)
        j = j+1
    plt.tight_layout()

In [ ]:
# creating list of products based on category
seating = list(product_data[product_data['CATEGORY'] == 'Seating']['PRODUCT_ID'])
tables = list(product_data[product_data['CATEGORY'] == 'Tables']['PRODUCT_ID'])
accessories = list(product_data[product_data['CATEGORY'] == 'Accessories']['PRODUCT_ID']) 

In [ ]:
product_plots(accessories)

In [ ]:
product_plots(seating)

In [ ]:
product_plots(tables)

#### Inferences:
- No increasing/decreasing trend for the sale of products over time
- No seasonal patterns seen on individual product sale
- Products by same manufaturer have similar patterns (spikes and drops).


### Dealer, Season and Product Sales
Is there a pattern of sales in a store? Eg: Holiday season with more sales or School season showing a increasing trend on sales

In [ ]:
# Randomly selecting 5 store ID
stores_plot = random.sample(list(store_data['STORE_ID']), 5)

In [ ]:
#creating dictionary with store number as keys
# for each store, calculate sum of units sold per week
d = {store: train[train['STORE_ID'] == store].groupby(['WEEK_END_DATE'])['UNITS'].sum() for store in stores_plot}

In [ ]:
plt.figure(figsize=(30,10))

fig, axs = plt.subplots(5, 1, figsize = (15, 15), dpi=300)
j = 0
for store in d.keys():
    sns.lineplot(x = d[store].index, y = d[store],ax = axs[j])
    j = j+1

#### Infereneces:
For the randomly selected store numbers, we can see that there is no pattern in the plot. The same was repeated for a number of stores and the data showed no increasing or decreasing trend or seasonality. 

### Featured or Displayed Product and Product sale

If the product is featured at the Dealer, does it have an impact on the sales? For example: Featured Products with attractive offers will have higher sales or Sales will be more for products with in-store promotion

In [ ]:
def featured_plots(product_list):
    #dictionary storing Product_id and 'Featured' variable
    d_f = {product: 1000*train[train['PRODUCT_ID'] == product].groupby(['WEEK_END_DATE'])['FEATURE'].mean() for product in product_list}
    #dictionary storing PRODUCT_ID and Product Sales
    d = {product: train[train['PRODUCT_ID'] == product].groupby(['WEEK_END_DATE'])['UNITS'].sum() for product in product_list}
    
    
    fig, axs = plt.subplots(len(product_list), 1, figsize = (20, 20), dpi=300)
    j = 0
    for product in d.keys():
        # Manufacturer name and Descritption in title
        manu = product_data[product_data['PRODUCT_ID'] == product]['MANUFACTURER'].values[0]
        desc = product_data[product_data['PRODUCT_ID'] == product]['DESCRIPTION'].values[0]
        
        # plotting featured and sales values
        sns.lineplot(x = d_f[product].index, y = d_f[product],ax = axs[j]).set_title(str(manu)+str(" ")+str(desc), y=0.75, fontsize = 16)
        sns.lineplot(x = d[product].index, y = d[product],ax = axs[j]).set_title(str(manu)+str(" ")+str(desc), y=0.75, fontsize = 16)
        j = j+1

In [ ]:
product_list_f = list(product_data[product_data['CATEGORY'] == 'Tables']['PRODUCT_ID'])

In [ ]:
featured_plots(product_list_f)

#### Infereneces:
When the products are featured, the sales increase.

#### Does the in-store display also have a similar effect?

In [ ]:
def display_plots(product_list):
    d_d = {product: 1000*train[train['PRODUCT_ID'] == product].groupby(['WEEK_END_DATE'])['DISPLAY'].mean() for product in product_list}
    d = {product: train[train['PRODUCT_ID'] == product].groupby(['WEEK_END_DATE'])['UNITS'].sum() for product in product_list}
    fig, axs = plt.subplots(len(product_list), 1, figsize = (20, 20), dpi=300)
    j = 0
    for product in d.keys():
        manu = product_data[product_data['PRODUCT_ID'] == product]['MANUFACTURER'].values[0]
        desc = product_data[product_data['PRODUCT_ID'] == product]['DESCRIPTION'].values[0]
        sns.lineplot(x = d[product].index, y = d[product],ax = axs[j]).set_title(str(manu)+str(" ")+str(desc), y=0.75, fontsize = 16)
        sns.lineplot(x = d_d[product].index, y = d_d[product],ax = axs[j]).set_title(str(manu)+str(" ")+str(desc), y=0.75, fontsize = 16)
        j = j+1

In [ ]:
display_plots(product_list_f)

#### Inferences:
- It is evident that product sales are greatly affected by the display.
- For products on display, the sales are higher.

### Product price and Product Sales 

Finding whether the price of the product have an effect on sales?


In [ ]:
# creating list of products based on category

product_size_seating = store_product_data.loc[store_product_data['CATEGORY']=='Seating']
product_size_tables  = store_product_data.loc[store_product_data['CATEGORY']=='Tables']
product_size_accessories = store_product_data.loc[store_product_data['CATEGORY']=='Accessories'] 

In [ ]:
# scatter plot for base price and sales
plt.figure(figsize=(8,6))
plt.scatter(x = (product_size_seating['BASE_PRICE']), y = (product_size_seating['UNITS']))
plt.xlabel('BASE_PRICE', fontsize=12)
plt.ylabel('UNITS', fontsize=12)
plt.show()

In [ ]:
# scatter plot for base price and sales
plt.figure(figsize=(8,6))
plt.scatter(x = (product_size_tables['BASE_PRICE']), y = (product_size_tables['UNITS']))
plt.xlabel('BASE_PRICE', fontsize=12)
plt.ylabel('UNITS', fontsize=12)
plt.show()

In [ ]:
# scatter plot for base price and sales
plt.figure(figsize=(8,6))
plt.scatter(x = (product_size_accessories['BASE_PRICE']), y = (product_size_accessories['UNITS']))
plt.xlabel('BASE_PRICE', fontsize=12)
plt.ylabel('UNITS', fontsize=12)
plt.show()

#### Inferences:
- For seatings, items with lower price show a higher sale. 
- Tables have higher sale for medium priced items. 
- Accessories show higher sale in high price cateogries with some outliers.

### Brand and Product sales

Finding if there is a trend of high sales on a particular brand

In [ ]:
tables = list(product_data[product_data['CATEGORY'] == 'Tables']['PRODUCT_ID'])
seating = list(product_data[product_data['CATEGORY'] == 'Seating']['PRODUCT_ID'])
accessories = list(product_data[product_data['CATEGORY'] == 'Accessories']['PRODUCT_ID'])

In [ ]:
plt.figure(figsize=(20,6))
ax = sns.boxplot(x="PRODUCT_ID", y="BASE_PRICE", data=train[train['PRODUCT_ID'].isin(tables)])
product_data[product_data['PRODUCT_ID'].isin(tables)]

In [ ]:
plt.figure(figsize=(20,6))
ax = sns.boxplot(x="MANUFACTURER", y="UNITS", data=store_product_data[store_product_data['PRODUCT_ID'].isin(tables)])

In [ ]:
plt.figure(figsize=(20,6))
ax = sns.boxplot(x="MANUFACTURER", y="UNITS", data=store_product_data[store_product_data['PRODUCT_ID'].isin(seating)])

In [ ]:
plt.figure(figsize=(20,6))
ax = sns.boxplot(x="MANUFACTURER", y="UNITS", data=store_product_data[store_product_data['PRODUCT_ID'].isin(accessories)])

#### Inference
- There is no evidence on high sales for particular brand

### Store location (State) and product sales

Finding if there is a significant difference in the product sales for different regions? 

    - Use Store Location: Is there a trend between a store/dealer location(State) and the number of order/units sold

In [ ]:
grouped_weekly_sales = store_product_data.groupby(['WEEK_END_DATE','STORE_ID'])['UNITS'].sum().reset_index()

grouped_weekly_sales = grouped_weekly_sales.merge(store_data, how = 'left', left_on = 'STORE_ID', right_on = 'STORE_ID')

grouped_weekly_sales = grouped_weekly_sales.sort_values(by = 'ADDRESS_STATE_PROV_CODE')

state = (store_data[['ADDRESS_STATE_PROV_CODE','STORE_ID']].sort_values(by ='ADDRESS_STATE_PROV_CODE'))['STORE_ID']

In [ ]:
plt.figure(figsize=(50,15))

ax=sns.boxplot(x="STORE_ID",y="UNITS",data=grouped_weekly_sales, hue ='ADDRESS_STATE_PROV_CODE', order =state)
plt.xticks(rotation=45)

#### Inferences:
- Mostly the number of units is higher for Ohio (considering individual stores)

### Dealer Store Size and Product sales

Finding whether store size increase product sales

In [ ]:
store_agg_data = train.groupby(['STORE_ID'])['UNITS'].sum().reset_index()
merged_store_data = store_data.merge(store_agg_data, how = 'left', left_on = 'STORE_ID', right_on = 'STORE_ID')

In [ ]:
state_oh = merged_store_data.loc[merged_store_data['ADDRESS_STATE_PROV_CODE'] == 'OH']
state_tx = merged_store_data.loc[merged_store_data['ADDRESS_STATE_PROV_CODE'] == 'TX']

sns.distplot(state_oh['SALES_AREA_SIZE_NUM'], hist=False,color= 'dodgerblue', label= 'OHIO')
sns.distplot(state_tx['SALES_AREA_SIZE_NUM'], hist=False,  color= 'orange', label= 'TEXAS')

In [ ]:
sns.scatterplot(x = (state_oh['SALES_AREA_SIZE_NUM']), y = (state_oh['UNITS']))
sns.scatterplot(x = (state_tx['SALES_AREA_SIZE_NUM']), y = (state_tx['UNITS']))

#### Inference
- Size of Store: Stores with larger area would have more sales

*************

# 6. Data Preprocessing

Data Preprocessing is a technique that is used to convert the raw data into a clean data set.

## PREPROCESSING: CATEGORICAL FEATURES

- Find out and impute, if we have missing values in the categorical features.
- Remove the features which do not add much information
- Choose an encoding scheme to convert categorical feature into numeric.

### TRAIN DATA SET:  
Weekly Sales Data contains the following categorical variables:
 - STORE_NUM
 - UPC
 - FEATURE
 - DISPLAY
    

In [ ]:
# check for the null values in the categorical features
train[['STORE_ID', 'PRODUCT_ID', 'FEATURE', 'DISPLAY']].isna().sum()

#### Inferences:
***No Null Values*** 
 -  STORE_ID  - No changes required as it is a key and will be used to merge tables later.
 -  PRODUCT_ID - No changes required as it is a key and will be used to merge tables later.
 -  FEATURE    - No Preprocessing Required, as the values are already 0 or 1 in the data set.
 -  DISPLAY    - No Preprocessing Required, as the values are already 0 or 1 in the data set.

**Create new column `WEEKOFYEAR` to capture the year & week number. This will be useful during prediction**

In [ ]:
train.WEEK_END_DATE = pd.to_datetime(train.WEEK_END_DATE)
train['weeknum']=train['WEEK_END_DATE'].dt.week
train['year'] = pd.DatetimeIndex(train['WEEK_END_DATE']).year
train['WEEKOFYEAR'] =train[['year','weeknum']].dot([100,1])

train= train.drop(columns=['weeknum'])
train= train.drop(columns=['year'])


### PRODUCT DATA SET :
Product Data has the following categorical variables:
- PRODUCT_ID
- DESCRIPTION
- MANUFACTURER
- CATEGORY
- SUB_CATEGORY
- PRODUCT_WEIGHT_LB

In [ ]:
product_data.dtypes

In [ ]:
# check for the null values in the categorical features
product_data[['PRODUCT_ID', 'DESCRIPTION', 'MANUFACTURER', 'CATEGORY', 'SUB_CATEGORY', 'PRODUCT_WEIGHT_LB']].isna().sum()

- `DESCRIPTION` - In the description, we have category, subcategory and size of the product and these are already present in the other features as well. So, We will drop this feature as it will not add much value to the model.
- `MANUFACTURER`, `CATEGORY`, `SUB_CATEGORY`- As, there is no order in the given categories, so we will change this as numerical variables. 

In [ ]:
# drop the DESCRIPTION FEATURE
product_data = product_data.drop(columns= ['DESCRIPTION'])

### STORE DATA SET :
Store Data has the following categorical variables:
- STORE_ID
- STORE_NAME
- ADDRESS_CITY_NAME
- ADDRESS_STATE_PROV_CODE
- MSA_CODE
- SEG_VALUE_NAME

In [ ]:
# check for the null values

store_data[['STORE_ID', 'STORE_NAME', 'ADDRESS_CITY_NAME', 'ADDRESS_STATE_PROV_CODE', 'MSA_CODE', 'SEG_VALUE_NAME']].isnull().sum()

   - STORE_ID - No changes required as it is a key and will be used to merge files later.
   - STORE_NAME - Since, Out of 76 different stores we have 72 unique store names. Store name contains some location information of the store which we have in the form of address city name and state.
   - ADDRESS_CITY_NAME - Since, Out of 76 different stores we have 51 unique address city names, So we will drop this feature due to high cardinality
   - ADDRESS_STATE_PROV_CODE, MSA_CODE - As, there is no order in the given categories. So, we can convert this as numerical variable using categorization.
   - SEG_VALUE_NAME - Stores segments are divided into 3 categories: upscale, mainstream and value. This field has no major impact on the sales and therefore, we can leave the column as is for now and analyse during model fitting process.

In [ ]:
# drop store name and address
store_data = store_data.drop(columns=['STORE_NAME', 'ADDRESS_CITY_NAME'])

In [ ]:
# columns of the updated data
store_data.columns

## PREPROCESSING: NUMERICAL FEATURES

- Check and impute the missing values in the numerical features.
- Check for the outliers and treat them.

### TRAIN DATA SET

WEEKLY SALES DATA  has the following numerical features

    - BASE_PRICE
    - UNITS (Target)

In [ ]:
# check the null values for the numerical features
train[[ 'BASE_PRICE', 'UNITS']].isna().sum()

No Nulls present.

In [ ]:
# scatter plot for UNITS variable
# sort the target variable and scatter plot to see if it has some outliers or not.  

%matplotlib notebook
plt.figure(figsize=(8,6))
plt.scatter(x = range(train.shape[0]), y = np.sort(train['UNITS'].values))
plt.xlabel('Index', fontsize=12)
plt.ylabel('Units Sold', fontsize=12)
plt.show()

In [ ]:
#Find the number of data points where units are more than 750, as the plot shows sparesely plotted for those values
train['UNITS'][train.UNITS > 750].shape[0]

#### Inferences:
There are a some points above where UNITS are more than 750 and there number is only 21. Considering 750 as the cutoff as only less number of plotting is present above 750.<br>
So, we can remove them as there number is only 21 and will not affect the data and these will act as a noise to our model.


In [ ]:
# remove the valures where UNITS are more than 750
train = train[~(train.UNITS > 750)]

### PRODUCT DATA SET

There are no numerical features in product data set

### STORE DATA SET

STORE DATA has the following numerical features
- PARKING_SPACE_QTY
- SALES_AREA_SIZE_NUM
- AVG_WEEKLY_ORDERS

In [ ]:
# check for the null values
store_data[['PARKING_SPACE_QTY', 'SALES_AREA_SIZE_NUM', 'AVG_WEEKLY_ORDERS']].isna().sum()

PARKING_SPACE_QTY - Check its correlation with the SALES_AREA_SIZE_NUM

In [ ]:
# check correlation
store_data[['PARKING_SPACE_QTY','SALES_AREA_SIZE_NUM']].corr()

#### Inferences:
Since the correlation of the PARKING_SPACE_QTY with SALES_AREA_SIZE_NUM is high so we can drop this column as it will not add much value to the model

In [ ]:
# drop the column
store_data = store_data.drop(columns=['PARKING_SPACE_QTY'])

### SAVE THE UPDATED FILES

In [ ]:
train.to_csv('data/updated_train_data.csv',index=False)
product_data.to_csv('data/updated_product_data.csv',index=False)
store_data.to_csv('data/updated_store_data.csv',index=False)

***********

# Loading data

In [ ]:
updated_product = pd.read_csv("data/updated_product_data.csv")
updated_store= pd.read_csv("data/updated_store_data.csv")
updated_train = pd.read_csv("data/updated_train_data.csv")

In [ ]:
updated_product.head()

In [ ]:
updated_train.head()

In [ ]:
updated_store.head()

In [ ]:
# Merging THe three filles t be one file

final_file = (pd.merge(updated_product, updated_train, on='PRODUCT_ID'))
final_file.head()

In [ ]:
All_final_file = (pd.merge(final_file, updated_store, how = 'left', on = 'STORE_ID'))
All_final_file.head() 

In [ ]:
All_final_file.isnull().any(axis=0)

In [ ]:
# convert to datetime
All_final_file.WEEK_END_DATE = pd.to_datetime(All_final_file.WEEK_END_DATE)

In [ ]:
All_final_file.to_csv('data/merged_data.csv', index=False)